<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#imports" data-toc-modified-id="imports-0.0.1"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>imports</a></span></li></ul></li></ul></li><li><span><a href="#Connect-to-local-Neo4J-database" data-toc-modified-id="Connect-to-local-Neo4J-database-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Connect to local Neo4J database</a></span><ul class="toc-item"><li><span><a href="#graph-creation-in-Py2Neo" data-toc-modified-id="graph-creation-in-Py2Neo-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>graph creation in Py2Neo</a></span><ul class="toc-item"><li><span><a href="#clean-nulls-from-member-dict" data-toc-modified-id="clean-nulls-from-member-dict-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>clean nulls from member dict</a></span></li></ul></li><li><span><a href="#Graph-population-methods" data-toc-modified-id="Graph-population-methods-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Graph population methods</a></span></li></ul></li><li><span><a href="#Querying" data-toc-modified-id="Querying-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Querying</a></span></li></ul></div>

### imports

In [1]:
import py2neo
from py2neo import Database
from py2neo import Graph, Node, Relationship
from py2neo import NodeMatcher, RelationshipMatcher
import neoconfig
import pickle


In [2]:
def get_industries(mems): # pass list-list-dict-dict data
    industries = [] # returns list of unique industries
    for chamber in mems: # this is awfully nested
        for member in chamber: # dict
            for k, v in member.items():
                if k!='party' and k!='name' and k!='id':
                    if k not in industries:
                        industries.append(k)
    return industries

def add_industry_nodes(G, industries): # pass graph, list of industries
    for industry in industries:
        G.add_node(str(industry), size=0, ind=industry)
        print(type(industry), type(G.node[industry]))
    return G # returns modified graph

def pickle_obj(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)
def unpickle_obj(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)
    
    


- NodeMatcher seems to create a copy of the node (for editing purposes)
- editing matched_node is like editing a copy, not the original object

In [3]:
def populate_graph(G, mems, industries): # add member nodes + edges
    edges = []
    ec=0
    for chamber in mems:
        for c, member in enumerate(chamber):
            if len(member)!=0:
                try:
                    mem_node = G.add_node(member['name'], size=0)
                    total_mem_size = 0
                    print(c)
                    for k, v in member.items():
                        if k!='party' and k!='name' and k!='id': # industry dict
                            G.add_edge(member['name'], v['name'], weight=v['total'])
#                             G[member['name']][v['name']]['party'] = member['party']
                            total_mem_size += int(v['total']) # update member's total bribery $
                            G.node[v['name']]['size'] += int(v['total']) # update industry's total bribery $
                    G.node[member['name']]['size'] = total_mem_size # reassign mem_node size
                    G.node[member['name']]['party'] = member['party']
                    G.node[member['name']]['name'] = member['name'] # give a name property
                except:
                    ec+=1 # count errors to ensure data integrity
    print(f'{ec} errors')
    return G

In [3]:
# members in json format
mems = unpickle_obj('json_data')

In [4]:
industries = get_industries(mems)

# Connect to local Neo4J database

In [163]:
db = py2neo.Database() # instantiate using default bolt port

In [164]:
g = py2neo.Graph(host='localhost', auth = ('neo4j', neoconfig.password))

the logic of py2neo seems to be:
- create transaction, the 'subgraph'
    - create Nodes or Relationships, assign placeholder variable names
    - transaction.create(placeholder_name) for anything you want to add
    - tx.commit() to push the subgraph up to the connected Neo db

## graph creation in Py2Neo

In [22]:
industries[2]

'Agricultural Services/Products'


- Py2Neo's Database.Transaction.commit() presents an issue updating new edge properties.
- it commits a subgraph to the larger Database, but:
            - "Any entities in subgraph that are already bound to remote entities will remain unchanged, those which are not will become bound to their newly-created counterparts"
- so since our edges are going to be programatically updated, we can't use Py2Neo's RelationshipMatcher to update the main graph
    - fix: stop mucking about with a wrapper and just use a Cypher statement
        - SET relationship.property = value  will instantiate a new property if not exists, and update if exists
    - cypher investigations: i can't use my beloved f-strings for a multiline query, because cypher uses the same {curly braces} that f-strings do.
        - but the % formatting works with '''multiline''', so no worries
            - later: %s, %d doesn't play nice with cypher interpretations.
            - dan used string concat + \backslash at the end of each line, but that gave me problems. putting strings on new lines inside ()tuple worked, though.
            - turns out Py2Neo has graph.run() which is designed to fix this sort of thing. uses $var_name substitution, pass them in the arguments. another example of 'read the docs, then keep reading'

### clean nulls from member dict

In [147]:
def clean_mems(mems):
    for ch, chamber in enumerate(mems):
        c=0
        print(f'starting length: {len(chamber)}')
        for mem in chamber:
            if not mem: 
                chamber.remove(mem) # throw away the null
                c+=1
            if mem['name'] = 
        if ch==0: cham = 'House'
        else: cham = 'Senate'
        print(f'removed {c} nulls from {cham}. length is now {len(chamber)}')
    return mems
mems = clean_mems(mems)

starting length: 436
removed 0 nulls from House. length is now 436
starting length: 99
removed 0 nulls from Senate. length is now 99


In [ ]:
# miscella

## Graph population methods

In [173]:
def update_node_weight(g, node_name, node_weight): # finds node by name, updates weight
    query = ('MATCH (n {name:$name_})' 
    'SET n.weight=$weight_')
    g.evaluate(query, name_=node_name, weight_=node_weight)
    return g

def add_industries(g, industries): # populates graph with Industry nodes
    tx = g.begin()
    for c, indu in enumerate(industries):
        n = Node('Industry', name=indu, weight=0) # weight is total money given
        tx.create(n)
        print(f'created {c+1} industries . . . ')
    tx.commit() # push industry nodes to local DB
    return g

def graphify(g, mems, industries):
    ''' args: p2neo graph, congressmembers_list_(dicts), industries_list_(strings)
    returns: p2neo graph obj.  also commits changes to local neo4j database'''
    g = add_industries(g, industries) # add industry nodes
    ind_totals = dict(zip(industries, [0]*len(industries))) # to keep track of industry totals    
    nmatch = NodeMatcher(g) # we could probably just query weight of all edges in Neo & SET it there...x
    print(f'created {len(industries)} industry nodes')
#     tx = g.begin() # can one tx handle all the data at once? let's find out
    for ch, chamber in enumerate(mems): 
        for c, mem in enumerate(chamber): # mem is dict
            print(f'starting {c+1} {mem["name"]}')
            tx = g.begin() # new tx for each node+all its edges
            if ch == 0: chamb = 'House'
            else: chamb = 'Senate'
                
            money_received = 0 # total $ for each member
            member_name = mem['name']
            part = mem['party']
            mem_node = Node(part, name=member_name, party=mem['party'],
                           cid=mem['id'], chamber=chamb, weight=0)
            tx.create(mem_node) # create member node
            tx.commit()
            print(f'created Node:  {member_name}')
                # create all edges from member node to industry nodes
            for key, val in mem.items(): # if key is str 'Agriculture', val is dict of that member's agri donation
                if key!='name' and key!='party' and key!='id': # industry donations
                    tx = g.begin() # new tx for each iteration, because reassigning var names
                    ind_name = val['name']
                    donation = int(val['total'])
                    print(f'adding Relationship: {ind_name}---{member_name}')
                    money_received += donation
                    ind_node = nmatch.match('Industry', name=ind_name).first() # match target industry node
                    bribe = Relationship(ind_node, 'DONATION', mem_node, weight=donation) # link industry node to this member's donation
                    tx.create(bribe) # add one edge to graph
                    ind_totals[ind_name] += donation # add to total for industry
                    tx.commit()
#                     print(f' linked {ind_name} to {mem_name} by {donation}')
            # done adding member's edges. update mem_node total $ weight
            # transactions are handled within this update method
            g = update_node_weight(g, member_name, money_received)
#             tx.commit()
#             print(f"{c}  {mem['name']} has received {money_receieved} in total")
    
    for key, val in ind_totals.items(): # update using ind_totals
        name = key
        total = val
        g = update_node_weight(g, name, total)
    print('[o] updated all industry node totals')
    return g # why not
                  
                  
                  

In [174]:
print(f'{len(mems[0]), len(mems[1])}  {len(industries)}  ') # good to go

(436, 99)  82  


In [175]:
g

<Graph database=<Database uri='bolt://localhost:7687' secure=False user_agent='py2neo/4.3.0 neobolt/1.7.15 Python/3.7.4-final-0 (darwin)'> name='data'>

In [176]:
g = graphify(g, mems, industries)

created 1 industries . . . 
created 2 industries . . . 
created 3 industries . . . 
created 4 industries . . . 
created 5 industries . . . 
created 6 industries . . . 
created 7 industries . . . 
created 8 industries . . . 
created 9 industries . . . 
created 10 industries . . . 
created 11 industries . . . 
created 12 industries . . . 
created 13 industries . . . 
created 14 industries . . . 
created 15 industries . . . 
created 16 industries . . . 
created 17 industries . . . 
created 18 industries . . . 
created 19 industries . . . 
created 20 industries . . . 
created 21 industries . . . 
created 22 industries . . . 
created 23 industries . . . 
created 24 industries . . . 
created 25 industries . . . 
created 26 industries . . . 
created 27 industries . . . 
created 28 industries . . . 
created 29 industries . . . 
created 30 industries . . . 
created 31 industries . . . 
created 32 industries . . . 
created 33 industries . . . 
created 34 industries . . . 
created 35 industries .

adding Relationship: Health Professionals---Jim Banks (R)
adding Relationship: Republican/Conservative---Jim Banks (R)
adding Relationship: Insurance---Jim Banks (R)
adding Relationship: Securities & Investment---Jim Banks (R)
starting 13 Lou Barletta (R)
created Node:  Lou Barletta (R)
adding Relationship: Retired---Lou Barletta (R)
adding Relationship: Republican/Conservative---Lou Barletta (R)
adding Relationship: Leadership PACs---Lou Barletta (R)
adding Relationship: Real Estate---Lou Barletta (R)
adding Relationship: Lawyers/Law Firms---Lou Barletta (R)
adding Relationship: Building Materials & Equipment---Lou Barletta (R)
adding Relationship: General Contractors---Lou Barletta (R)
adding Relationship: Health Professionals---Lou Barletta (R)
adding Relationship: Misc Manufacturing & Distributing---Lou Barletta (R)
adding Relationship: Oil & Gas---Lou Barletta (R)
starting 14 Andy Barr (R)
created Node:  Andy Barr (R)
adding Relationship: Securities & Investment---Andy Barr (R)
ad

adding Relationship: Health Services/HMOs---Diane Black (R)
adding Relationship: Lobbyists---Diane Black (R)
adding Relationship: Misc Manufacturing & Distributing---Diane Black (R)
adding Relationship: Hospitals/Nursing Homes---Diane Black (R)
adding Relationship: Telecom Services---Diane Black (R)
adding Relationship: Beer, Wine & Liquor---Diane Black (R)
adding Relationship: Women's Issues---Diane Black (R)
adding Relationship: Tobacco---Diane Black (R)
adding Relationship: Air Transport---Diane Black (R)
adding Relationship: Misc Finance---Diane Black (R)
starting 29 Marsha Blackburn (R)
created Node:  Marsha Blackburn (R)
adding Relationship: Retired---Marsha Blackburn (R)
adding Relationship: Republican/Conservative---Marsha Blackburn (R)
adding Relationship: Real Estate---Marsha Blackburn (R)
adding Relationship: Leadership PACs---Marsha Blackburn (R)
adding Relationship: Securities & Investment---Marsha Blackburn (R)
adding Relationship: Health Professionals---Marsha Blackburn 

adding Relationship: Health Services/HMOs---Larry Bucshon (R)
adding Relationship: Leadership PACs---Larry Bucshon (R)
adding Relationship: Electric Utilities---Larry Bucshon (R)
adding Relationship: Lawyers/Law Firms---Larry Bucshon (R)
adding Relationship: Automotive---Larry Bucshon (R)
adding Relationship: Misc Energy---Larry Bucshon (R)
adding Relationship: Real Estate---Larry Bucshon (R)
adding Relationship: Insurance---Larry Bucshon (R)
starting 46 Ted Budd (R)
created Node:  Ted Budd (R)
adding Relationship: Leadership PACs---Ted Budd (R)
adding Relationship: Republican/Conservative---Ted Budd (R)
adding Relationship: Retired---Ted Budd (R)
adding Relationship: Commercial Banks---Ted Budd (R)
adding Relationship: Real Estate---Ted Budd (R)
adding Relationship: Securities & Investment---Ted Budd (R)
adding Relationship: Insurance---Ted Budd (R)
adding Relationship: Candidate Committees---Ted Budd (R)
adding Relationship: Finance/Credit Companies---Ted Budd (R)
adding Relationship

adding Relationship: Transportation Unions---Judy Chu (D)
adding Relationship: Human Rights---Judy Chu (D)
starting 63 David Cicilline (D)
created Node:  David Cicilline (D)
adding Relationship: Lawyers/Law Firms---David Cicilline (D)
adding Relationship: Retired---David Cicilline (D)
adding Relationship: Real Estate---David Cicilline (D)
adding Relationship: Building Trade Unions---David Cicilline (D)
adding Relationship: Securities & Investment---David Cicilline (D)
adding Relationship: TV/Movies/Music---David Cicilline (D)
adding Relationship: Democratic/Liberal---David Cicilline (D)
adding Relationship: Misc Manufacturing & Distributing---David Cicilline (D)
adding Relationship: Telecom Services---David Cicilline (D)
adding Relationship: Human Rights---David Cicilline (D)
starting 64 Katherine Clark (D)
created Node:  Katherine Clark (D)
adding Relationship: Lawyers/Law Firms---Katherine Clark (D)
adding Relationship: Pharmaceuticals/Health Products---Katherine Clark (D)
adding Rel

...

adding Relationship: Health Professionals---Susan Collins (R)
adding Relationship: Lobbyists---Susan Collins (R)
adding Relationship: Real Estate---Susan Collins (R)
adding Relationship: Misc Manufacturing & Distributing---Susan Collins (R)
adding Relationship: Pro-Israel---Susan Collins (R)
starting 18 Chris Coons (D)
created Node:  Chris Coons (D)
adding Relationship: Lawyers/Law Firms---Chris Coons (D)
adding Relationship: Securities & Investment---Chris Coons (D)
adding Relationship: Leadership PACs---Chris Coons (D)
adding Relationship: Lobbyists---Chris Coons (D)
adding Relationship: Pharmaceuticals/Health Products---Chris Coons (D)
adding Relationship: TV/Movies/Music---Chris Coons (D)
adding Relationship: Commercial Banks---Chris Coons (D)
adding Relationship: Real Estate---Chris Coons (D)
adding Relationship: Electronics Mfg & Equip---Chris Coons (D)
adding Relationship: Insurance---Chris Coons (D)
starting 19 Bob Corker (R)
created Node:  Bob Corker (R)
adding Relationship: L

starting 35 Lindsey Graham (R)
created Node:  Lindsey Graham (R)
adding Relationship: Lawyers/Law Firms---Lindsey Graham (R)
adding Relationship: Securities & Investment---Lindsey Graham (R)
adding Relationship: Retired---Lindsey Graham (R)
adding Relationship: Real Estate---Lindsey Graham (R)
adding Relationship: Leadership PACs---Lindsey Graham (R)
adding Relationship: Pro-Israel---Lindsey Graham (R)
adding Relationship: Health Professionals---Lindsey Graham (R)
adding Relationship: Lobbyists---Lindsey Graham (R)
adding Relationship: Business Services---Lindsey Graham (R)
adding Relationship: Misc Finance---Lindsey Graham (R)
starting 36 Chuck Grassley (R)
created Node:  Chuck Grassley (R)
adding Relationship: Retired---Chuck Grassley (R)
adding Relationship: Securities & Investment---Chuck Grassley (R)
adding Relationship: Leadership PACs---Chuck Grassley (R)
adding Relationship: Lawyers/Law Firms---Chuck Grassley (R)
adding Relationship: Lobbyists---Chuck Grassley (R)
adding Relati

adding Relationship: Securities & Investment---Angus King (I)
adding Relationship: Real Estate---Angus King (I)
adding Relationship: Leadership PACs---Angus King (I)
adding Relationship: Health Professionals---Angus King (I)
adding Relationship: Insurance---Angus King (I)
adding Relationship: Pro-Israel---Angus King (I)
adding Relationship: Education---Angus King (I)
starting 53 Amy Klobuchar (D)
created Node:  Amy Klobuchar (D)
adding Relationship: Lawyers/Law Firms---Amy Klobuchar (D)
adding Relationship: Retired---Amy Klobuchar (D)
adding Relationship: Securities & Investment---Amy Klobuchar (D)
adding Relationship: Women's Issues---Amy Klobuchar (D)
adding Relationship: TV/Movies/Music---Amy Klobuchar (D)
adding Relationship: Leadership PACs---Amy Klobuchar (D)
adding Relationship: Education---Amy Klobuchar (D)
adding Relationship: Real Estate---Amy Klobuchar (D)
adding Relationship: Business Services---Amy Klobuchar (D)
adding Relationship: Lobbyists---Amy Klobuchar (D)
starting 5

adding Relationship: Real Estate---David Perdue (R)
adding Relationship: Leadership PACs---David Perdue (R)
adding Relationship: Lawyers/Law Firms---David Perdue (R)
adding Relationship: Insurance---David Perdue (R)
adding Relationship: Misc Finance---David Perdue (R)
adding Relationship: Food & Beverage---David Perdue (R)
adding Relationship: Business Services---David Perdue (R)
adding Relationship: Health Professionals---David Perdue (R)
starting 71 Gary Peters (D)
created Node:  Gary Peters (D)
adding Relationship: Lawyers/Law Firms---Gary Peters (D)
adding Relationship: Retired---Gary Peters (D)
adding Relationship: Securities & Investment---Gary Peters (D)
adding Relationship: Leadership PACs---Gary Peters (D)
adding Relationship: Real Estate---Gary Peters (D)
adding Relationship: Insurance---Gary Peters (D)
adding Relationship: Environment---Gary Peters (D)
adding Relationship: Education---Gary Peters (D)
adding Relationship: Health Professionals---Gary Peters (D)
adding Relation

adding Relationship: Environment---Jon Tester (D)
adding Relationship: Health Professionals---Jon Tester (D)
adding Relationship: Business Services---Jon Tester (D)
adding Relationship: Insurance---Jon Tester (D)
starting 89 John Thune (R)
created Node:  John Thune (R)
adding Relationship: Retired---John Thune (R)
adding Relationship: Securities & Investment---John Thune (R)
adding Relationship: Lobbyists---John Thune (R)
adding Relationship: Real Estate---John Thune (R)
adding Relationship: Insurance---John Thune (R)
adding Relationship: Lawyers/Law Firms---John Thune (R)
adding Relationship: Leadership PACs---John Thune (R)
adding Relationship: Health Professionals---John Thune (R)
adding Relationship: Air Transport---John Thune (R)
adding Relationship: Telecom Services---John Thune (R)
starting 90 Thom Tillis (R)
created Node:  Thom Tillis (R)
adding Relationship: Retired---Thom Tillis (R)
adding Relationship: Securities & Investment---Thom Tillis (R)
adding Relationship: Real Estat

- okay, we crashed at 

In [140]:
# mems_backup = mems

starting length: 439
removed 3 nulls from House. length is now 436
starting length: 100
removed 1 nulls from Senate. length is now 99


In [146]:
mems = mems_backup

# Querying

In [ ]:
# i'm doing a lot of this in the blog
# https://medium.com/@mark.s.cleverley/graphing-congressional-lobbying-549e9e5ecc87
# https://medium.com/@mark.s.cleverley/querying-congressional-lobbying-in-neo4j-25cecc8a7ce8
# https://medium.com/@mark.s.cleverley/algorithmic-detection-in-congressional-lobbying-fdb4a8e98ada?sk=4c0a5cfd8c3052820755242fc4d772cc